# Item 27: Prefer Public Attributes Over Private Ones

- In Python, there are only two types of atrribute visibility for a class's attributes: *public* and *private*.

In [2]:
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
        
    def get_private_field(self):
        return self.__private_field

- Public attributes can be accessed by anyone using the dot operator on the object.

In [3]:
foo = MyObject()
assert foo.public_field == 5

- Private fields are specified by prefixing an attribute's name with a double underscore. They can be accessed directly by methods of the containing class.

In [4]:
assert foo.get_private_field() == 10

- Directly accessing private fields from outside the class raises an exception.

In [5]:
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

- Class methods also have access to private attributes because they are declared within the surrounding class block.

In [6]:
class MyOtherObject(object):
    def __init__(self):
        self.__private_field = 71
        
    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field
    
bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

- As you'd expect with private fields, a subclass can't access its parent class's private fields.

In [8]:
class MyParentObject(object):
    def __init__(self):
        self.__private_field = 71
        
    
class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field
    
baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

- The private attribute behavior is implemented with a simple transformation of the attribute name. When the Python compiler sees private attribute access in methods like MychildObject.get\_private\_field, it translates \_\_private\)field to \_\_private\_field was only defined in MyParentObject.\_\_init\_\_, meaning the private attribute's real name is \_MyParentObject\_\_private\_field. Accessing the parent's private attribute from the child class fails simply because the transformed attribute name doesn't match.

- Knowing this scheme, you can easily access the private attributes of any class, from a subclass or externally, without asking form permission.

In [9]:
assert baz._MyParentObject__private_field == 71

- Why doesn't the syntax for private attributes actually enforce strict visibility? The simplest answer is one often-quoted motto of Python:"We are all consenting adults here." Python programmers believe that the benefits of being open outweigh the downsides of being closed.

- Beyondthat, having the ability to hook language features like attribute access enables you to mess around with the internals of objects whenever you wish. If you can do that, what is the value of Python trying to prevent private attributes access otherwise?

- TO minimize the damage of accessing internals unknowingly, PYthon programmers follow a naming convention defined in the style guide. Fields prefixed by a single underscore (like \_protected\_field) are *protected*, meaning external users of the class should proceed with caution.

- However, many programmers who are new to Python use private fields to indicate an internal API that shouldn't be accessed by subclasses or externally.

In [10]:
class MyClass(object):
    def __init__(self, value):
        self.__value = value
        
    def get_value(self):
        return str(self.__value)
    
foo = MyClass(5)
assert foo.get_value() == '5'

- This is wrong approach. Inevitably someone, including you, will want to subclass your class to add new behavior to work around deficiencies in existing mehtods (like above, how MyClass.get\_value always returns a string). By choosing private attributes, you're only making subclass overrides and extensions cumbersome and brittle. Your potential subclassers will still access the private fields when they absolutely need to do so.

In [11]:
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)
    

foo = MyIntegerSubclass(5)
assert foo.get_value() == 5

- But if the class hierarchy changes beneath you, these classes will break because the private references are no longer valid. Here, the MyIntegerSubclass class's immediate parent, MyClass, has had another parent class added called MyBaseClass:

In [12]:
class MyBaseClass(object):
    def __init__(self, value):
        self.__value = value
    #...
    
class MyClass(MyBaseClass):
    #....
    
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

IndentationError: expected an indented block (<ipython-input-12-7d55dfe3b51f>, line 9)

- The \_\_value attribute is now assigned in the MyBaseClass parent class, not the MyClass parent. That causes the private variable reference self.\_MyClass\_\_value to break in MyIntegerSubclass.

In [ ]:
foo = MyIntegerSubclass(5)
foo.get_value()

- In general, it's better to err on the side of allowing subclasses to do more by using protected attributes. Document each protected field and explain which are internal APIs available to subclasses and which should be left alone entirely. This is as much advice to other programmers as it is guidance for your future self on how to extend your own code safely.

In [13]:
class MyClass(object):
    def __init__(self, value):
        # This stores the user-supplied value for the object.
        # It should be coercible to a string. Once assigned for
        # the object it should be treated as immutable.
        self._value = value

- The only time to seriously consider using private attributes is when you're worried about naming conflicts with subclasses. This problem occurs when a child class unwitiingly defines an attribute that was already defined by its parent class.

In [17]:
class ApiClass(object):
    def __init__(self):
        self._value = 5
        
    def get(self):
        return self._value
    
    
class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' # conflicts
        
a = Child()
print(a.get(), 'and', a._value, 'should be different')

hello and hello should be different


- This is primarily a concern with classes that are part of a public API; the subclasses are out of your control, so you can't refactor to fix the problem. Such a conflict is especially possible with attribute names that are very common (like value). To reduce the risk of this happening, you can use a private attribute in the parent class to ensure there are no attribute names that overlap with child classes.

In [21]:
class ApiClass(object):
    def __init__(self):
        self.__value = 5
        
    def get(self):
        return self.__value
    
    
class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' # OK
        
a = Child()
print(a.get(), 'and', a._value, 'are different')

5 and hello are different


## Things To Remember

- Private attributes aren't rigorously enforced by the Python compiler.
- Use documentation of protected fields to guide subclasses instead of trying to force access control with private attributes.
- Only consider using private attributes to avoid naming conflicts with subclasses that are ouy of your control

# Item 28: Inherits from collections.abc for Custom Container Types

- Much of programming in Python is defining classes that contain data and describing how such objects relate to each other. Every Python class is a container of some kind, encapsulating attributes and functionality together. Python also provides built-in container types for managing data: lists, tuples, sets, and dictionaries.

- When you're designing classes for simple use cases like sequences, it's natural that you'd want to subclass Python's built-in list type directly. For example, say you want to create your own custom list type that has additional methods for counting the frequency of its members.

In [22]:
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
        
    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts

- By subclassing list, you get all of list's standard functionality and preserve the semantics familiar to all Python programmers. Your additional methods can add any custom behaviors you need.

In [23]:
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print('Length is', len(foo))
foo.pop()
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

Length is 7
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


- Now, imagine you want to provide an object that feels like a list, allowing indexing, but isn't list subclass. For example, say you want to provide sequence semantics (like list or tuple) for a binary tree class.

In [24]:
class BinaryNode(object):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

- How do you make this act like a sequence type? Python implements its container behaviors with instance methods that have special names. When you access a sequence item by index.

- it will be interpreted as:

In [26]:
bar = [1, 2, 3]
bar[0]
bar.__getitem__(0)

1

- To make BinaryNode class act like a sequence, you can provide a custom implementation of \_\_getitem\_\_ that traverses the object tree depth first.

In [29]:
class IndexableNode(BinaryNode):
    def _search(self, count, index):
        # ..
        # Returns (found, count)
        pass
        
    def __getitem__(self, index):
        found, _ = self._search(0, index)
        if not found:
            raise IndexError('Index out of range')
        return found.value

- You can construct your binary tree as usual.

In [30]:
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(
            6, right=IndexableNode(7))),
    right=IndexableNode(
        15, left=IndexableNode(11)))

In [31]:
type(tree)

__main__.IndexableNode

In [32]:
tree.left

In [34]:
tree[0]

TypeError: 'NoneType' object is not iterable

- The problem is that implementing \_\_getitem\_\_ isn't enough to provide all of the sequence semantics you'd expect.

In [35]:
len(tree)

TypeError: object of type 'IndexableNode' has no len()

- The len built-in function requires another special method named \_\_len\_\_ that must have an implementation for your custom sequence type.

In [ ]:
class SequenceNode(IndexableNode):
    def __len__(self):
        _, count = self._search(0, None)
        return count
    
    
tree = SequenceNode(
    #...
)
print('Tree has $d nodes' % len(tree))

- Unfortunately, this stiill isn't enough. Also missing are the count and index methods that a Python programmer would expect to see on a sequence like list or uple. Defining your own container types is much harder than it looks.

- To avoid this difficulty throughout the Python universe, the built-in collections.abc module defines a set of abstract base classes that provide all of the typical methods for each container type. When you subclass from these abstract base classes and forget to implement require methods, the module will tell you something is wrong.

In [36]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType()

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

- When you do implement all of the methods required by an abstract base class, as I did above with SequenceNode, it will provide all of the additional methods like index and count for free.

In [37]:
class BetterNode(SequenceNode, Sequence):
    pass

tree = BetterNode(
    #...
)

print('Index of 7 is', tree.index(7))

NameError: name 'SequenceNode' is not defined

- The benefit of using these abstract base classes is even greater for more complex types like set and MutableMapping, which have a large number of special methods that need to be implemented to match Python conventions.

## Things to Remember

- Inherit directly from Python's container types (like list or dict) for simple use cases.
- Beware of the large number of methods required to implement custom container types correctly.
- Have your suctom container types inherit from the interfaces defined in collections.abc to ensure that your classes match required interfaces and behaviors.

# Metaclasses and Attributes

- Metaclasses are often mentionedin lists of Python's features, but few understand what they accomplish in practice. The name *metaclass* vaguely implies a concept above and beyond a class. Simply put, metaclasses let you intercept Python's class statement and provide special behavior each time a class is defined.

- Similarly mysterious and powerful are Python's built-in features for dynamically customizing attribute accesses. Along with Python's object-oriented constructs, these facilities provide wonderful tools to ease the transition from simple classes to complex ones.

- However, with these powers come many pitfalls. Dynamic attributes enable you to override objects and cause unexpected side effects. Meataclasses can create extremely bizarre behaviors that are unapproachable to newcomers. It's important that you follow the *rule of least surprise* and only use these mechanisms to implement well-understood idioms.

# Item 29: Use Plain Attributes Instead of Get and Set Methods

- Programmers coming to Python from other languages may naturally try to implement explicit getter and setter methods in their classes.

In [38]:
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms

- Using these setters and getters is simple, but it's not Pythonic.

In [39]:
r0 = OldResistor(50e3)
print('Before: %5r' % r0.get_ohms())
r0.set_ohms(10e3)
print('After: %5r' % r0.get_ohms())

Before: 50000.0
After: 10000.0


- Such methods are especially clumsy for operations like incrementing in place.

In [40]:
r0.set_ohms(r0.get_ohms() + 5e3)

- These utility methods do help define the interface for your class, making it easier to encapsulate functionality, validate usage, and define boundaries. Those are important goals when designing a class to ensure you don't break callers as your class evolves over time.

- In Python, however, you almost never need to implement explicit setter or getter methods. Instead, you should always start your implementations with simple public attributes.

In [41]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
r1 = Resistor(50e3)
r1.ohms = 10e3

- These make operations like incrementing in place natural and clear.

In [42]:
r1.ohms += 5e3

- Later, if you decide you need special behavior when an attribute is set, you can migrate to the @property decorator and its corresponding setter attribute. Here, I define a new subclass of Resistor that lets me vary the current by assigning the voltage property. Note that in order to work properly the name of both the setter and getter methods must match the intended property name.

In [43]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

- Now, assigning the voltage property will run the voltage setter method, updating the current property of the object to match.

In [44]:
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After: %5r amps' % r2.current)

Before:     0 amps
After:  0.01 amps


- Specifying a setter on a property also lets you perform type checking and validation on values passed to your class. Here, I define a class that all resistance values are above zero ohms:

In [49]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms

- Assigning an invalid resistance to the attribute raises an exception.

In [50]:
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: 0.000000 ohms must be > 0

- This happens because BoundedResistance.\_\_init\_\_ calls Resistor.\_\_init\_\_, which assigns self.ohms = -5. That assignment causes the @ohms.setter method from BoundedResistance to be called, immediately running the validation code before object construction has completed. 

- You can even use @property to make attributes from parent classes immutable.

In [51]:
class FixedResistance(Resistor):
    #...
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

- Trying to assign to the property after construction raises an exception.

In [52]:
r4 = FixedResistance(1e3)
r4.ohms = 2e3

AttributeError: Can't set attribute

- The biggest shortcoming of @property is that the methos for an attribute can only be shared by subclasses. Unrelated classes can't share the same implementation. However, Python also supports *descriptors* that enable reusable property logic and many other use cases.

- Finally, when you use @property methods to implement setters and getters, be sure that the behavior you implement is not surprising. For example, don't set other attributes in getter property methods.

In [53]:
class MysteriouResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    # ...
    

- This leads to extremely bizarre behavior

In [ ]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print('Before: %5r' % r7.voltage)
r7.ohms
print('After: %5r' % r7.voltage)

- The best policy is to modify related object state in @property.setter methods. Be sure to avoid any other side effects the caller may not expect beyond the object, such as importing modules dynamically, running slow helper functions, or making expensive database queries. Users of your class will expect its attributes to be like any other Python object: quick and easy. Use normal methods to do anythings more complex or slow.

## Things To Remember

- Define new class interfaces using simple public attributes, and avoid set and get methods.
- Use @property to define special behavior when attributes are accessed on your objects, if necessary.
- Follow the rule of least surprise and avoid weird side effects in your @property methods.
- Ensure that @property methods are fast; do slow or complex work using normal methods.

# Item 30: Consider @preperty Instead of Refactoring Attributes

- The built-in @property decorator makes it easy for simple accesses of an instance's attributes to act smarter. One advanced but common use of @property is transitioning what was once a simple numerical attribute into an on-the-fly calculation. This is extremely helpful because it lets you migrate all exsiting uasge of a class to have new behavior without rewriting any of the call sites. It also provieds an important stopgap for improving your interfaces over time.

- For example, say you want to implement a leaky bucket quota using plain Python objects. Here, the Bucket class represents how much quota remains and the duration for which the quota will be available.

In [1]:
class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0
        
    def __repr__(self):
        return 'Bucket(quota=%d)' % self.quota

- The leaky bucket algorithm works by ensuring that, whenever the bucket is filled, the amount of quota does not carry over from one period to the next.

In [2]:
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

- Each time a quota consumer wants to do something, it first must ensure that it can deduct the amount of quota it needs to use.

In [3]:
def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

- To use this class, first I fill the bucket.